In [1]:
# Import Necessary Libraries
import os
import json
import logging
import polars as pl
from google.cloud import firestore, bigquery, storage
from google.api_core.exceptions import GoogleAPICallError
from firebase_admin import credentials, firestore
from dotenv import load_dotenv
from google.auth.exceptions import RefreshError
from google.oauth2 import service_account
import hashlib

# Load environment variables
load_dotenv()

project_id = os.getenv("GOOGLE_FIRESTORE_PROJECT_ID")
database_name = os.getenv("GOOGLE_FIRESTORE_DATABASE_NAME")
collection_name = os.getenv("GOOGLE_FIRESTORE_COLLECTION_NAME")
bigquery_project_id = os.getenv("GOOGLE_BIGQUERY_PROJECT_ID")
bigquery_dataset_id = os.getenv("GOOGLE_BIGQUERY_DATASET_NAME")
gcp_credentials_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

# Configure Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Ensure the Credentials file exists
print(f"Current working directory: {os.getcwd()}")

# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print(f"Parent directory: {parent_dir}")

# Path to the credentials file
gcp_credentials_path = os.path.join(parent_dir, "config/gcp_credentials.json")
print(f"Credentials path: {gcp_credentials_path}")

# Path to query file
query_file_path = os.path.join(parent_dir, "database/create_tables.sql")
print(f"Query file path: {query_file_path}")

# Check if the credentials file exists
if not os.path.isfile(gcp_credentials_path):
    print(f"Error: Credentials file not found at {gcp_credentials_path}")
    exit(1)

# Print if the file contains valid JSON
try:
    with open(gcp_credentials_path, 'r') as f:
        json_data = json.load(f)
        logging.info("Credentials file contains valid JSON.")
except json.JSONDecodeError:
    logging.error(f"Credentials file is not valid JSON.")
    exit(1)

# Ensure credentials file exists
if not os.path.exists(gcp_credentials_path):
    logging.error(f"Failed to initialize Firestore and BigQuery: File {gcp_credentials_path} was not found.")
    exit(1)

# Initialize BigQuery Client
try:
    client = bigquery.Client.from_service_account_json(gcp_credentials_path)
    logging.info("BigQuery client initialized successfully!")
except RefreshError as e:
    logging.error(f"Failed to initialize BigQuery due to authentication error: {e}")
    exit(1)
except Exception as e:
    logging.error(f"Failed to initialize BigQuery: {e}")
    exit(1)

# Get table names from BigQuery dataset
try:
    query = f"SELECT table_name FROM `{bigquery_project_id}.{bigquery_dataset_id}.INFORMATION_SCHEMA.TABLES`"
    tables = client.query(query).to_dataframe()
    table_names = tables["table_name"].tolist()
    logging.info(f"Tables in BigQuery dataset: {table_names}")
except GoogleAPICallError as e:
    logging.error(f"BigQuery API error: {e}")
    exit(1)

# Initialize Firestore Client
try:
    credentials = service_account.Credentials.from_service_account_file(gcp_credentials_path)
    db = firestore.Client(project=project_id, database=database_name, credentials=credentials)
    logging.info("Firestore client initialized successfully!")
except RefreshError as e:
    logging.error(f"Failed to initialize Firestore due to authentication error: {e}")
    exit(1)
except Exception as e:
    logging.error(f"Failed to initialize Firestore: {e}")
    exit(1)

# Read 5 random documents from Firestore collection
random_documents = db.collection(collection_name).limit(2000).get()

2025-03-03 23:30:57,454 - INFO - Credentials file contains valid JSON.
2025-03-03 23:30:57,491 - INFO - BigQuery client initialized successfully!


Current working directory: c:\Users\RameezRassdeen\Documents\shipment-tracking-project\notebooks
Parent directory: c:\Users\RameezRassdeen\Documents\shipment-tracking-project
Credentials path: c:\Users\RameezRassdeen\Documents\shipment-tracking-project\config/gcp_credentials.json
Query file path: c:\Users\RameezRassdeen\Documents\shipment-tracking-project\database/create_tables.sql


2025-03-03 23:31:01,863 - INFO - Tables in BigQuery dataset: ['DocumentReferences', 'EquipmentEvent', 'Seals', 'Address', 'Vessel', 'Facility', 'TransportCall', 'EventLocation']
2025-03-03 23:31:01,890 - INFO - Firestore client initialized successfully!


In [2]:
# Convert documents to JSON format
json_data = [doc.to_dict() for doc in random_documents]

# Create a Polars DataFrame from the JSON data
df = pl.DataFrame(json_data)

# Print schema to verify structure
print(df.schema)

Schema({'eventCreatedDateTime': String, 'eventClassifierCode': String, 'equipmentEventTypeCode': String, 'documentReferences': List(Struct({'documentReferenceType': String, 'documentReferenceValue': String})), 'eventLocation': Struct({'facilityCodeListProvider': String, 'longitude': String, 'unlocationCode': String, 'address': Struct({'stateRegion': String, 'streetNumber': String, 'street': String, 'country': String, 'floor': String, 'city': String, 'postCode': String, 'name': String}), 'facilityCode': String, 'latitude': String, 'locationName': String}), 'eventID': String, 'seals': List(Struct({'sealType': String, 'sealNumber': String, 'sealSource': String})), 'equipmentReference': String, 'transportCall': Struct({'facilityCodeListProvider': String, 'vessel': Struct({'vesselCallSignNumber': String, 'vesselOperatorCarrierCode': String, 'vesselOperatorCarrierCodeListProvider': String, 'vesselIMONumber': String, 'vesselFlag': String, 'vesselName': String}), 'carrierServiceCode': String, 

In [3]:
df

eventCreatedDateTime,eventClassifierCode,equipmentEventTypeCode,documentReferences,eventLocation,eventID,seals,equipmentReference,transportCall,eventDateTime,eventType
str,str,str,list[struct[2]],struct[7],str,list[struct[3]],str,struct[9],str,str
"""2024-06-20T08:12:59.656226""","""ACT""","""UNLOAD""","[{""BL"",""b510ce35-82e2-418c-be24-23537f216701""}]","{""SMDG"",""-93.41878"",""XYZ123"",{""N/A"",""21"",""Main Street"",""UK"",""8F"",""København"",""7462"",""Henrik""},""XYZ"",""-25.481948"",""Port Terminal""}","""00006ba7-4179-4b4d-a8b9-7020e4…","[{""STANDARD"",""a09ba28e-0391-4964-92f3-fca907667ae3"",""CUSTOMS""}]","""APZU2386924""","{""SMDG"",{""NCVV"",""MAEU"",""NMFTA"",""9323208"",""DE"",""Sea Guardian""},""FE1"",""0c582809-f5b3-4641-bf50-c5f724ed11f0"",""SEA"",""3925N"",""PORT"",""XYZ"",""5256S""}","""2024-06-20T08:12:59.656226""","""EQUIPMENT"""
"""2024-05-13T08:12:59.276063""","""EST""","""UNLOAD""","[{""BL"",""ca6d86d6-e81a-4813-8e25-8a69e93cdaba""}]","{""SMDG"",""11.40495"",""XYZ123"",{""Region B"",""157"",""Main Street"",""UK"",""5F"",""København"",""1757"",""Henrik""},""XYZ"",""71.330214"",""Port Terminal""}","""00052089-881b-4774-9131-cef97a…","[{""STANDARD"",""52fcfbe8-87da-4ba8-9e35-526fc2bfdf7e"",""CUSTOMS""}]","""APZU3785346""","{""SMDG"",{""NCVV"",""MAEU"",""NMFTA"",""9487386"",""DE"",""Ocean Explorer""},""FE1"",""d2ab6d03-4b1c-4733-9e5a-8d9b94095a7f"",""SEA"",""5221N"",""PORT"",""ADT"",""6348S""}","""2024-05-13T08:12:59.276063""","""EQUIPMENT"""
"""2024-04-18T08:13:00.066059""","""EST""","""ARRIVAL""","[{""BL"",""e231f39f-837c-4c5c-a55d-ebbd1092398c""}]","{""SMDG"",""64.146556"",""XYZ123"",{""Region B"",""101"",""Main Street"",""Germany"",""10F"",""Berlin"",""6471"",""Henrik""},""ADT"",""24.834957"",""Port Terminal""}","""0005ee8f-8e98-4015-bd9c-5165a1…","[{""STANDARD"",""35e8b99c-061a-4c18-8e22-f8630355935c"",""CARRIER""}]","""APZU7792130""","{""SMDG"",{""NCVV"",""MAEU"",""NMFTA"",""9948579"",""DE"",""Ocean Explorer""},""FE1"",""3ac370c1-9a95-41a1-8e4f-f6a0e18b8fe7"",""SEA"",""4516N"",""PORT"",""PQR"",""7942S""}","""2024-04-18T08:13:00.066059""","""EQUIPMENT"""
"""2024-11-16T08:12:59.493765""","""EST""","""LOAD""","[{""BL"",""51770229-8fb7-4874-b277-05916fbd81d4""}]","{""SMDG"",""-155.880456"",""XYZ123"",{""Region A"",""13"",""Main Street"",""Norway"",""9F"",""London"",""6673"",""Henrik""},""PQR"",""68.375913"",""Port Terminal""}","""0007d2dd-d688-4f65-bd85-1e01c8…","[{""STANDARD"",""1fc4b733-c92f-431e-b4b8-5cfdb3ea2027"",""CUSTOMS""}]","""APZU3279567""","{""SMDG"",{""NCVV"",""MAEU"",""NMFTA"",""9606882"",""DE"",""Ocean Explorer""},""FE1"",""01c92dab-eb1b-4ad8-b2af-67b557af4dbf"",""SEA"",""9859N"",""PORT"",""ADT"",""5245S""}","""2024-11-16T08:12:59.493765""","""EQUIPMENT"""
"""2024-03-13T08:12:59.213024""","""EST""","""DEPARTURE""","[{""BL"",""bb6d6842-690d-491f-a639-1ab7de72e4d7""}]","{""SMDG"",""104.079924"",""XYZ123"",{""N/A"",""63"",""Main Street"",""Denmark"",""6F"",""København"",""3242"",""Henrik""},""ADT"",""-55.313004"",""Port Terminal""}","""0007f87a-8748-466c-bd12-d1df68…","[{""STANDARD"",""2c6e9d52-b123-4de9-8be4-e5bc4f5fa4cc"",""CARRIER""}]","""APZU7556078""","{""SMDG"",{""NCVV"",""MAEU"",""NMFTA"",""9423653"",""DE"",""Ocean Explorer""},""FE1"",""a6d4b08a-9757-4330-ab01-2c1e34674f28"",""SEA"",""9461N"",""PORT"",""PQR"",""2259S""}","""2024-03-13T08:12:59.213024""","""EQUIPMENT"""
…,…,…,…,…,…,…,…,…,…,…
"""2024-04-24T08:13:00.138821""","""ACT""","""ARRIVAL""","[{""BL"",""d65364bf-1203-4bdd-b817-7d9698479d32""}]","{""SMDG"",""-152.540061"",""XYZ123"",{""Region A"",""145"",""Main Street"",""Germany"",""6F"",""Paris"",""8101"",""Henrik""},""ADT"",""-5.784531"",""Port Terminal""}","""191ddf5c-9798-4c3a-a7ee-a2ba42…","[{""STANDARD"",""d7543a25-f4b4-46fe-9a45-bd9d3844e376"",""CARRIER""}]","""APZU9937254""","{""SMDG"",{""NCVV"",""MAEU"",""NMFTA"",""9626280"",""DE"",""Wave Rider""},""FE1"",""48dd6650-2596-4a9f-ae2a-3008ade45edf"",""SEA"",""2823N"",""PORT"",""PQR"",""7251S""}","""2024-04-24T08:13:00.138821

In [4]:
# Flatten the main event fields
df_events = df.select([
    pl.col("eventID").alias("eventID"),
    pl.col("eventCreatedDateTime").alias("eventCreatedDateTime"),
    pl.col("eventClassifierCode").alias("eventClassifierCode"),
    pl.col("equipmentEventTypeCode").alias("equipmentEventTypeCode"),
    pl.col("equipmentReference").alias("equipmentReference"),
    pl.col("eventDateTime").alias("eventDateTime"),
    pl.col("eventType").alias("eventType")
])

# Parse string columns as datetime (including fractional seconds)
df_events = df_events.with_columns([
    pl.col("eventCreatedDateTime").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%.f").alias("eventCreatedDateTime"),
    pl.col("eventDateTime").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%.f").alias("eventDateTime")
])

# Convert datetime columns to ISO 8601 strings (without fractional seconds)
df_events = df_events.with_columns([
    pl.col("eventCreatedDateTime").dt.strftime("%Y-%m-%dT%H:%M:%S").alias("eventCreatedDateTime"),
    pl.col("eventDateTime").dt.strftime("%Y-%m-%dT%H:%M:%S").alias("eventDateTime")
])

# Explode document references with eventID initially
df_document_references = (
    df.select([
        pl.col("eventID"),
        pl.col("documentReferences").explode().alias("document_reference")
    ])
    .group_by("eventID", maintain_order=True)
    .agg([
        pl.col("document_reference").struct.field("documentReferenceType").alias("documentReferenceType"),
        pl.col("document_reference").struct.field("documentReferenceValue").alias("documentReferenceValue"),
        pl.col("document_reference").cum_count().cast(pl.Int64).alias("documentIndex")
    ])
    .explode(["documentReferenceType", "documentReferenceValue", "documentIndex"])
)

# Compute SHA-256 hash once using documentReferenceType and documentReferenceValue
df_document_references = df_document_references.with_columns([
    pl.concat_str([pl.col("documentReferenceType"), pl.col("documentReferenceValue")], separator="_")
    .map_elements(lambda x: hashlib.sha256(x.encode()).hexdigest(), return_dtype=pl.Utf8).alias("hashed_col")
])

# Generate documentReferenceID
df_document_references = df_document_references.with_columns([
    pl.col("hashed_col").map_elements(lambda x: f"{x[:4]}-{x[4:8]}-{x[8:12]}-{x[12:16]}", return_dtype=pl.Utf8).alias("documentReferenceID")
])

# Remove hashed_col column
df_document_references = df_document_references.drop("hashed_col")

# Join with df_events to add documentReferenceID
df_events = df_events.join(
    df_document_references.select(["eventID", "documentReferenceID"]),
    on="eventID",
    how="left"
)

# Ensure uniqueness in df_document_references (Avoid duplicates) after removing eventID
df_document_references = df_document_references.drop("eventID").unique(subset=["documentReferenceID"])

# Reorder columns to ensure documentReferenceID is the first column in df_document_references
df_document_references = df_document_references.select([
    "documentReferenceID",
    *[col for col in df_document_references.columns if col != "documentReferenceID"]
])

# Extract eventLocation
df_event_location = df.select([
    pl.col("eventID").alias("eventID"),
    pl.col("eventLocation").struct.field("latitude").cast(pl.Float64).alias("latitude"),
    pl.col("eventLocation").struct.field("longitude").cast(pl.Float64).alias("longitude"),
    pl.col("eventLocation").struct.field("unlocationCode").alias("unlocationCode")
])

# Extract eventTransportCall
df_event_transport_call = df.select([
    pl.col("eventID").alias("eventID"),
    pl.col("transportCall").struct.field("transportCallID").alias("transportCallID"),
    pl.col("transportCall").struct.field("carrierServiceCode").alias("carrierServiceCode"),
    pl.col("transportCall").struct.field("exportVoyageNumber").alias("exportVoyageNumber"),
    pl.col("transportCall").struct.field("importVoyageNumber").alias("importVoyageNumber"),
    pl.col("transportCall").struct.field("modeOfTransport").alias("modeOfTransport")
])

# Extract eventAddress
df_event_address = df.select([
    pl.col("eventID").alias("eventID"),
    pl.col("eventLocation").struct.field("address").struct.field("stateRegion").alias("stateRegion"),
    pl.col("eventLocation").struct.field("address").struct.field("streetNumber").alias("streetNumber"),
    pl.col("eventLocation").struct.field("address").struct.field("street").alias("street"),
    pl.col("eventLocation").struct.field("address").struct.field("country").alias("country"),
    pl.col("eventLocation").struct.field("address").struct.field("floor").alias("floor"),
    pl.col("eventLocation").struct.field("address").struct.field("city").alias("city"),
    pl.col("eventLocation").struct.field("address").struct.field("postCode").alias("postCode"),
    pl.col("eventLocation").struct.field("address").struct.field("name").alias("name")
])

# Compute SHA-256 hash once using stateRegion, streetNumber, street, country, floor, city, postCode
df_event_address = df_event_address.with_columns([
    pl.concat_str([
        pl.col("stateRegion"),
        pl.col("streetNumber"),
        pl.col("street"),
        pl.col("country"),
        pl.col("floor"),
        pl.col("city"),
        pl.col("postCode")
    ], separator="_")
    .map_elements(lambda x: hashlib.sha256(x.encode()).hexdigest(), return_dtype=pl.Utf8).alias("hashed_col")
])

# Generate addressID using stateRegion, streetNumber, street, country, floor, city, postCode
df_event_address = df_event_address.with_columns([
    pl.col("hashed_col").map_elements(lambda x: f"{x[:4]}-{x[4:8]}-{x[8:12]}-{x[12:16]}", return_dtype=pl.Utf8).alias("addressID")
])

# Remove hashed_col column
df_event_address = df_event_address.drop("hashed_col")


# Extract eventFacility
df_event_facility = df.select([
    pl.col("eventID").alias("eventID"),
    pl.col("eventLocation").struct.field("facilityCodeListProvider").alias("facilityCodeListProvider"),
    pl.col("eventLocation").struct.field("facilityCode").alias("facilityCode"),
    pl.col("transportCall").struct.field("facilityTypeCode").alias("facilityTypeCode")
])

# Compute SHA-256 hash once facilityID using facilityCodeListProvider, facilityCode & facilityTypeCode
df_event_facility = df_event_facility.with_columns([
    pl.concat_str([
        pl.col("facilityCodeListProvider"),
        pl.col("facilityCode"),
        pl.col("facilityTypeCode")
    ], separator="_")
    .map_elements(lambda x: hashlib.sha256(x.encode()).hexdigest(), return_dtype=pl.Utf8).alias("hashed_col")
])

# Generate facilityID using facilityCodeListProvider, facilityCode & facilityTypeCode
df_event_facility = df_event_facility.with_columns([
    pl.col("hashed_col").map_elements(lambda x: f"{x[:4]}-{x[4:8]}-{x[8:12]}-{x[12:16]}", return_dtype=pl.Utf8).alias("facilityID")
])

# Remove hashed_col column
df_event_facility = df_event_facility.drop("hashed_col")


# Extract vessel
df_vessel = df.select([
    pl.col("eventID").alias("eventID"),
    pl.col("transportCall").struct.field("vessel").struct.field("vesselCallSignNumber").alias("vesselCallSignNumber"),
    pl.col("transportCall").struct.field("vessel").struct.field("vesselFlag").alias("vesselFlag"),
    pl.col("transportCall").struct.field("vessel").struct.field("vesselIMONumber").alias("vesselIMONumber"),
    pl.col("transportCall").struct.field("vessel").struct.field("vesselName").alias("vesselName"),
    pl.col("transportCall").struct.field("vessel").struct.field("vesselOperatorCarrierCode").alias("vesselOperatorCarrierCode"),
    pl.col("transportCall").struct.field("vessel").struct.field("vesselOperatorCarrierCodeListProvider").alias("vesselOperatorCarrierCodeListProvider")
])

# Compute SHA-256 hash once using vesselCallSignNumber, vesselFlag, vesselIMONumber, vesselName, vesselOperatorCarrierCode & vesselOperatorCarrierCodeListProvider
df_vessel = df_vessel.with_columns([
    pl.concat_str([
        pl.col("vesselCallSignNumber"),
        pl.col("vesselFlag"),
        pl.col("vesselIMONumber"),
        pl.col("vesselName"),
        pl.col("vesselOperatorCarrierCode"),
        pl.col("vesselOperatorCarrierCodeListProvider")
    ], separator="_")
    .map_elements(lambda x: hashlib.sha256(x.encode()).hexdigest(), return_dtype=pl.Utf8).alias("hashed_col")
])

# Generate vesselID using vesselCallSignNumber, vesselFlag, vesselIMONumber, vesselName, vesselOperatorCarrierCode & vesselOperatorCarrierCodeListProvider
df_vessel = df_vessel.with_columns([
    pl.col("hashed_col").map_elements(lambda x: f"{x[:4]}-{x[4:8]}-{x[8:12]}-{x[12:16]}", return_dtype=pl.Utf8).alias("vesselID")
])

# Remove hashed_col column
df_vessel = df_vessel.drop("hashed_col")

# Join with df_transport_call to add vesselID
df_event_transport_call = df_event_transport_call.join(
    df_vessel.select(["eventID", "vesselID"]),
    on="eventID",
    how="left"
)

# Ensure uniqueness in df_vessel (Avoid duplicates) after removing eventID
df_vessel = df_vessel.drop("eventID").unique(subset=["vesselID"])

# Reorder columns to ensure vesselID is the first column in df_vessel
df_vessel = df_vessel.select([
    "vesselID",
    *[col for col in df_vessel.columns if col != "vesselID"]
])

# Join with df_event_transport_call to add facilityID
df_event_transport_call = df_event_transport_call.join(
    df_event_facility.select(["eventID", "facilityID"]),
    on="eventID",
    how="left"
)

# Join with df_event_location to add facilityID
df_event_location = df_event_location.join(
    df_event_facility.select(["eventID", "facilityID"]),
    on="eventID",
    how="left"
)

# Ensure uniqueness in df_event_facility (Avoid duplicates) after removing eventID
df_event_facility = df_event_facility.drop("eventID").unique(subset=["facilityID"])

# Reorder columns to ensure facilityID is the first column in df_event_facility
df_event_facility = df_event_facility.select([
    "facilityID",
    *[col for col in df_event_facility.columns if col != "facilityID"]
])

# Join with df_events to add addressID
df_event_location = df_event_location.join(
    df_event_address.select(["eventID", "addressID"]),
    on="eventID",
    how="left"
)

# Ensure uniqueness in df_event_address (Avoid duplicates) after removing eventID
df_event_address = df_event_address.drop("eventID").unique(subset=["addressID"])

# Reorder columns to ensure addressID is the first column in df_event_address
df_event_address = df_event_address.select([
    "addressID",
    *[col for col in df_event_address.columns if col != "addressID"]
])

# Join with df_events to add transportCallID (Not Generated as it already exists in the table)
df_events = df_events.join(
    df_event_transport_call.select(["eventID", "transportCallID"]),
    on="eventID",
    how="left"
)

# Ensure uniqueness in df_event_transport_call (Avoid duplicates) after removing event
df_event_transport_call = df_event_transport_call.drop("eventID").unique(subset=["transportCallID"])

# Reorder columns to ensure transportCallID is the first column in df_event_transport_call
df_event_transport_call = df_event_transport_call.select([
    "transportCallID",
    *[col for col in df_event_transport_call.columns if col != "transportCallID"]
])

# Compute SHA-256 hash once locationID using latitude, longitude, unlocationCode, facilityID & addressID
df_event_location = df_event_location.with_columns([
    pl.concat_str([
        pl.col("latitude"),
        pl.col("longitude"),
        pl.col("unlocationCode"),
        pl.col("facilityID"),
        pl.col("addressID")
    ], separator="_")
    .map_elements(lambda x: hashlib.sha256(x.encode()).hexdigest(), return_dtype=pl.Utf8).alias("hashed_col")
])

# Generate locationID using latitude, longitude, unlocationCode, facilityID & addressID
df_event_location = df_event_location.with_columns([
    pl.col("hashed_col").map_elements(lambda x: f"{x[:4]}-{x[4:8]}-{x[8:12]}-{x[12:16]}", return_dtype=pl.Utf8).alias("locationID")
])


# Remove hashed_col column
df_event_location = df_event_location.drop("hashed_col")

# Join with df_events to add locationID
df_events = df_events.join(
    df_event_location.select(["eventID", "locationID"]),
    on="eventID",
    how="left"
)

# Ensure uniqueness in df_event_location (Avoid duplicates) after removing eventID
df_event_location = df_event_location.drop("eventID").unique(subset=["locationID"])

# Reorder columns to ensure locationID is the first column in df_event_location
df_event_location = df_event_location.select([
    "locationID",
    *[col for col in df_event_location.columns if col != "locationID"]
])

# Explode seal with eventID initially
df_seals = (
    df.select([
        pl.col("eventID"),
        pl.col("seals").explode().alias("seal")
    ])
    .group_by("eventID", maintain_order=True)
    .agg([
        pl.col("seal").struct.field("sealNumber").alias("sealNumber"),
        pl.col("seal").struct.field("sealSource").alias("sealSource"),
        pl.col("seal").struct.field("sealType").alias("sealType"),
        pl.col("seal").cum_count().cast(pl.Int64).alias("sealIndex")
    ])
    .explode(["sealNumber", "sealSource", "sealType", "sealIndex"])
)


# Compute SHA-256 hash once sealID using sealNumber, sealSource, sealType with Truncate
df_seals = df_seals.with_columns([
    pl.concat_str([
        pl.col("sealNumber"),
        pl.col("sealSource"),
        pl.col("sealType")
    ], separator="_")
    .map_elements(lambda x: hashlib.sha256(x.encode()).hexdigest(), return_dtype=pl.Utf8).alias("hashed_col")
])

# Generate sealID using sealNumber, sealSource, sealType
df_seals = df_seals.with_columns([
    pl.col("hashed_col").map_elements(lambda x: f"{x[:4]}-{x[4:8]}-{x[8:12]}-{x[12:16]}", return_dtype=pl.Utf8).alias("sealID")
])

# Remove hash_col column
df_seals = df_seals.drop("hashed_col")

# Join with df_event to add sealID
df_events = df_events.join(
    df_seals.select(["eventID", "sealID"]),
    on="eventID",
    how="left"
)

# Ensure uniqueness in df_seals (Avoid duplicates) after removing eventID
df_seals = df_seals.drop("eventID").unique(subset=["sealID"])

# Reorder columns to ensure sealID is the first column in df_seals
df_seals = df_seals.select([
    "sealID",
    *[col for col in df_seals.columns if col != "sealID"]
])

##### Show results

In [5]:
# Set Polars display options
#pl.Config.set_tbl_hide_column_data_types(True)  # Hide data types

In [6]:
# Set the total width of the table display
#pl.Config.set_tbl_width_chars(400)  # Adjust this value as needed # Show full strings without truncation

In [7]:
print(df_events)

shape: (2_000, 11)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ eventID   ┆ eventCrea ┆ eventClas ┆ equipment ┆ … ┆ documentR ┆ transport ┆ locationI ┆ sealID   │
│ ---       ┆ tedDateTi ┆ sifierCod ┆ EventType ┆   ┆ eferenceI ┆ CallID    ┆ D         ┆ ---      │
│ str       ┆ me        ┆ e         ┆ Code      ┆   ┆ D         ┆ ---       ┆ ---       ┆ str      │
│           ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ str       ┆ str       ┆          │
│           ┆ str       ┆ str       ┆ str       ┆   ┆ str       ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 00006ba7- ┆ 2024-06-2 ┆ ACT       ┆ UNLOAD    ┆ … ┆ 1fc1-0abd ┆ 0c582809- ┆ d0ca-2b02 ┆ 50b9-403 │
│ 4179-4b4d ┆ 0T08:12:5 ┆           ┆           ┆   ┆ -c9ce-795 ┆ f5b3-4641 ┆ -3d89-004 ┆ 7-4cf2-d │
│ -a8b9-702 ┆ 9         ┆           ┆           ┆   ┆ 3         ┆ -bf50-

In [8]:
print(df_document_references)

shape: (2_000, 4)
┌─────────────────────┬───────────────────────┬─────────────────────────────────┬───────────────┐
│ documentReferenceID ┆ documentReferenceType ┆ documentReferenceValue          ┆ documentIndex │
│ ---                 ┆ ---                   ┆ ---                             ┆ ---           │
│ str                 ┆ str                   ┆ str                             ┆ i64           │
╞═════════════════════╪═══════════════════════╪═════════════════════════════════╪═══════════════╡
│ 56a8-555d-bbbf-75da ┆ BL                    ┆ b00c8e74-c2f9-43fb-8829-ebe0bc… ┆ 1             │
│ 10d4-f90e-2e91-1f99 ┆ BL                    ┆ 7812f3bf-ebdf-4334-a7ca-9e4c9c… ┆ 1             │
│ 5d12-6757-98da-3435 ┆ BL                    ┆ bf161518-8b6d-4fa8-bf2f-196c8c… ┆ 1             │
│ 1fdb-94ed-4a3c-abae ┆ BL                    ┆ 551b12fd-e42d-4633-9608-0fc93a… ┆ 1             │
│ 3315-f949-54cb-e0bd ┆ BL                    ┆ 96787606-e92e-4fe0-8a5b-8f1d70… ┆ 1             │
│ 

In [9]:
print(df_seals)

shape: (2_000, 5)
┌─────────────────────┬─────────────────────────────────┬────────────┬──────────┬───────────┐
│ sealID              ┆ sealNumber                      ┆ sealSource ┆ sealType ┆ sealIndex │
│ ---                 ┆ ---                             ┆ ---        ┆ ---      ┆ ---       │
│ str                 ┆ str                             ┆ str        ┆ str      ┆ i64       │
╞═════════════════════╪═════════════════════════════════╪════════════╪══════════╪═══════════╡
│ 8d1c-98fa-e937-ec97 ┆ a5834c89-0a7a-4e27-8f68-2c9064… ┆ CARRIER    ┆ STANDARD ┆ 1         │
│ f846-c0f4-50d6-bd95 ┆ 9632ce7d-ce15-4960-8733-afc2bd… ┆ CUSTOMS    ┆ STANDARD ┆ 1         │
│ f402-2ab0-8134-0e83 ┆ 27b2fe8e-42b4-4c05-8c68-c7662f… ┆ CUSTOMS    ┆ STANDARD ┆ 1         │
│ 652d-b110-133c-644f ┆ 6d26dc87-09e6-4f49-bb62-f10bc8… ┆ CARRIER    ┆ STANDARD ┆ 1         │
│ 54ab-6a26-4836-6042 ┆ aec112ff-418d-42a2-8b6d-1dad9b… ┆ CUSTOMS    ┆ STANDARD ┆ 1         │
│ …                   ┆ …                 

In [10]:
print(df_event_location)

shape: (2_000, 6)
┌──────────────────┬────────────┬─────────────┬────────────────┬─────────────────┬─────────────────┐
│ locationID       ┆ latitude   ┆ longitude   ┆ unlocationCode ┆ facilityID      ┆ addressID       │
│ ---              ┆ ---        ┆ ---         ┆ ---            ┆ ---             ┆ ---             │
│ str              ┆ f64        ┆ f64         ┆ str            ┆ str             ┆ str             │
╞══════════════════╪════════════╪═════════════╪════════════════╪═════════════════╪═════════════════╡
│ a0f6-d122-2b44-a ┆ 17.328797  ┆ 173.140922  ┆ XYZ123         ┆ ebe1-7dd2-8898- ┆ efe3-fc5c-2591- │
│ f60              ┆            ┆             ┆                ┆ 0f4f            ┆ 5034            │
│ 531c-68b7-ab65-1 ┆ 73.388155  ┆ -0.17207    ┆ XYZ123         ┆ 41d8-f6d8-11d2- ┆ bd2b-7df3-bccf- │
│ de0              ┆            ┆             ┆                ┆ cce1            ┆ 5ba1            │
│ f391-0b6b-4775-4 ┆ 22.622739  ┆ -134.810523 ┆ XYZ123         ┆ 41d8-f6d

In [11]:
print(df_event_facility)

shape: (3, 4)
┌─────────────────────┬──────────────────────────┬──────────────┬──────────────────┐
│ facilityID          ┆ facilityCodeListProvider ┆ facilityCode ┆ facilityTypeCode │
│ ---                 ┆ ---                      ┆ ---          ┆ ---              │
│ str                 ┆ str                      ┆ str          ┆ str              │
╞═════════════════════╪══════════════════════════╪══════════════╪══════════════════╡
│ 41d8-f6d8-11d2-cce1 ┆ SMDG                     ┆ XYZ          ┆ PORT             │
│ c8b1-05d3-2b7a-ee8e ┆ SMDG                     ┆ PQR          ┆ PORT             │
│ ebe1-7dd2-8898-0f4f ┆ SMDG                     ┆ ADT          ┆ PORT             │
└─────────────────────┴──────────────────────────┴──────────────┴──────────────────┘


In [12]:
print(df_event_address)

shape: (2_000, 9)
┌───────────────┬──────────────┬──────────────┬────────┬───┬───────┬───────────┬──────────┬────────┐
│ addressID     ┆ stateRegion  ┆ streetNumber ┆ street ┆ … ┆ floor ┆ city      ┆ postCode ┆ name   │
│ ---           ┆ ---          ┆ ---          ┆ ---    ┆   ┆ ---   ┆ ---       ┆ ---      ┆ ---    │
│ str           ┆ str          ┆ str          ┆ str    ┆   ┆ str   ┆ str       ┆ str      ┆ str    │
╞═══════════════╪══════════════╪══════════════╪════════╪═══╪═══════╪═══════════╪══════════╪════════╡
│ d86c-c0d2-635 ┆ Region A     ┆ 56           ┆ Main   ┆ … ┆ 8F    ┆ Berlin    ┆ 7611     ┆ Henrik │
│ 6-4130        ┆              ┆              ┆ Street ┆   ┆       ┆           ┆          ┆        │
│ c43d-1360-15c ┆ Region A     ┆ 60           ┆ Main   ┆ … ┆ 6F    ┆ Oslo      ┆ 8852     ┆ Henrik │
│ 2-e599        ┆              ┆              ┆ Street ┆   ┆       ┆           ┆          ┆        │
│ 9b20-c55f-6cc ┆ Region A     ┆ 188          ┆ Main   ┆ … ┆ 5F    ┆ Oslo

In [13]:
print(df_event_transport_call)

shape: (2_000, 7)
┌──────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┐
│ transportCal ┆ carrierServ ┆ exportVoyag ┆ importVoyag ┆ modeOfTrans ┆ vesselID    ┆ facilityID  │
│ lID          ┆ iceCode     ┆ eNumber     ┆ eNumber     ┆ port        ┆ ---         ┆ ---         │
│ ---          ┆ ---         ┆ ---         ┆ ---         ┆ ---         ┆ str         ┆ str         │
│ str          ┆ str         ┆ str         ┆ str         ┆ str         ┆             ┆             │
╞══════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╡
│ 7c263397-376 ┆ FE1         ┆ 5392S       ┆ 8483N       ┆ SEA         ┆ 242f-a54c-9 ┆ 41d8-f6d8-1 │
│ 3-4c84-8a9a- ┆             ┆             ┆             ┆             ┆ 12e-4b95    ┆ 1d2-cce1    │
│ 2ba85b…      ┆             ┆             ┆             ┆             ┆             ┆             │
│ a0edb3e2-040 ┆ FE1         ┆ 3619S       ┆ 6965N       ┆ SEA         ┆ 

In [14]:
print(df_vessel)

shape: (2_000, 7)
┌──────────────┬──────────────┬────────────┬─────────────┬─────────────┬─────────────┬─────────────┐
│ vesselID     ┆ vesselCallSi ┆ vesselFlag ┆ vesselIMONu ┆ vesselName  ┆ vesselOpera ┆ vesselOpera │
│ ---          ┆ gnNumber     ┆ ---        ┆ mber        ┆ ---         ┆ torCarrierC ┆ torCarrierC │
│ str          ┆ ---          ┆ str        ┆ ---         ┆ str         ┆ ode         ┆ odeListP…   │
│              ┆ str          ┆            ┆ str         ┆             ┆ ---         ┆ ---         │
│              ┆              ┆            ┆             ┆             ┆ str         ┆ str         │
╞══════════════╪══════════════╪════════════╪═════════════╪═════════════╪═════════════╪═════════════╡
│ b575-e710-f7 ┆ NCVV         ┆ DE         ┆ 9401968     ┆ King of the ┆ MAEU        ┆ NMFTA       │
│ 4b-ec99      ┆              ┆            ┆             ┆ Seas        ┆             ┆             │
│ ab4e-1cb6-aa ┆ NCVV         ┆ DE         ┆ 9640862     ┆ Sea         ┆ 

In [15]:
def upsert_to_bigquery(client, table_id, df, unique_key):
    """
    Upserts data into a BigQuery table.

    Args:
        client: BigQuery client.
        table_id: Full table ID in the format 'project_id.dataset_id.table_id'.
        df: Polars DataFrame containing the data to upsert.
        unique_key: The unique key column(s) to match rows for upsert.
    """
    # Convert Polars DataFrame to Pandas DataFrame (BigQuery client works with Pandas)
    pandas_df = df.to_pandas()

    # Create a temporary table ID
    temp_table_id = f"{table_id}_temp"

    # Load the data into a temporary table
    job_config = bigquery.LoadJobConfig(
        autodetect=False,  # Disable autodetect to ensure schema matches
        write_disposition="WRITE_TRUNCATE",  # Overwrite the temporary table
    )

    # Load the DataFrame into the temporary table
    job = client.load_table_from_dataframe(pandas_df, temp_table_id, job_config=job_config)
    job.result()  # Wait for the job to complete

    # Construct the MERGE SQL statement
    columns = pandas_df.columns.tolist()
    set_clause = ", ".join(
        [
            f"{col} = SAFE_CAST(temp.{col} AS TIMESTAMP)"
            if col in ["eventCreatedDateTime", "eventDateTime"]
            else f"{col} = temp.{col}"
            for col in columns
        ]
    )
    insert_columns = ", ".join(columns)
    insert_values = ", ".join(
        [
            f"SAFE_CAST(temp.{col} AS TIMESTAMP)"
            if col in ["eventCreatedDateTime", "eventDateTime"]
            else f"temp.{col}"
            for col in columns
        ]
    )

    merge_sql = f"""
    MERGE `{table_id}` AS target
    USING `{temp_table_id}` AS temp
    ON {' AND '.join([f'target.{key} = temp.{key}' for key in unique_key])}
    WHEN MATCHED THEN
        UPDATE SET {set_clause}
    WHEN NOT MATCHED THEN
        INSERT ({insert_columns}) VALUES ({insert_values})
    """

    # Execute the MERGE statement
    query_job = client.query(merge_sql)
    query_job.result()  # Wait for the job to complete

    # Delete the temporary table
    client.delete_table(temp_table_id, not_found_ok=True)

    logging.info(f"Upserted data into {table_id} successfully!")

In [16]:
# Upsert df_events into BigQuery
upsert_to_bigquery(
    client=client,
    table_id=f"{bigquery_project_id}.{bigquery_dataset_id}.EquipmentEvent",
    df=df_events,
    unique_key=["eventID"]
)

# Upsert df_document_references into BigQuery
upsert_to_bigquery(
    client=client,
    table_id=f"{bigquery_project_id}.{bigquery_dataset_id}.DocumentReferences",
    df=df_document_references,
    unique_key=["documentReferenceID"]
)

# Upsert df_event_location into BigQuery
upsert_to_bigquery(
    client=client,
    table_id=f"{bigquery_project_id}.{bigquery_dataset_id}.EventLocation",
    df=df_event_location,
    unique_key=["locationID"]
)

# Upsert df_event_transport_call into BigQuery
upsert_to_bigquery(
    client=client,
    table_id=f"{bigquery_project_id}.{bigquery_dataset_id}.TransportCall",
    df=df_event_transport_call,
    unique_key=["transportCallID"]
)

# Upsert df_event_address into BigQuery
upsert_to_bigquery(
    client=client,
    table_id=f"{bigquery_project_id}.{bigquery_dataset_id}.Address",
    df=df_event_address,
    unique_key=["addressID"]
)

# Upsert df_event_facility into BigQuery
upsert_to_bigquery(
    client=client,
    table_id=f"{bigquery_project_id}.{bigquery_dataset_id}.Facility",
    df=df_event_facility,
    unique_key=["facilityID"]
)

# Upsert df_vessel into BigQuery
upsert_to_bigquery(
    client=client,
    table_id=f"{bigquery_project_id}.{bigquery_dataset_id}.Vessel",
    df=df_vessel,
    unique_key=["vesselID"]
)

# Upsert df_seals into BigQuery
upsert_to_bigquery(
    client=client,
    table_id=f"{bigquery_project_id}.{bigquery_dataset_id}.Seals",
    df=df_seals,
    unique_key=["sealID"]
)

2025-03-03 23:31:22,959 - INFO - Upserted data into data-air-452306-e8.Logistics.EquipmentEvent successfully!
2025-03-03 23:31:32,538 - INFO - Upserted data into data-air-452306-e8.Logistics.DocumentReferences successfully!
2025-03-03 23:31:42,286 - INFO - Upserted data into data-air-452306-e8.Logistics.EventLocation successfully!
2025-03-03 23:31:51,487 - INFO - Upserted data into data-air-452306-e8.Logistics.TransportCall successfully!
2025-03-03 23:31:59,720 - INFO - Upserted data into data-air-452306-e8.Logistics.Address successfully!
2025-03-03 23:32:09,355 - INFO - Upserted data into data-air-452306-e8.Logistics.Facility successfully!
2025-03-03 23:32:22,619 - INFO - Upserted data into data-air-452306-e8.Logistics.Vessel successfully!
2025-03-03 23:32:30,655 - INFO - Upserted data into data-air-452306-e8.Logistics.Seals successfully!
